In [ ]:
pip install -U sentence-transformers

In [1]:
import os
import re

from sentence_transformers import SentenceTransformer, util



preprocess the input 

We take the input form a txt file titles "text.txt"

we read the txt file, and we split it into sentences.

In [2]:
directory= os.getcwd()
wordsPath= directory + "/text.txt"
with open(wordsPath, 'r') as file:
     contents = file.read()
contents= contents.lower()

sentences=re.split(r'[,.!?]+', contents)
sentences


["in today's meeting",
 ' we discussed a variety of issues affecting our department',
 ' the weather was unusually sunny',
 ' a pleasant backdrop to our serious discussions',
 ' we came to the consensus that we need to do better in terms of performance',
 ' sally brought doughnuts',
 ' which lightened the mood',
 " it's important to make good use of what we have at our disposal",
 ' during the coffee break',
 ' we talked about the upcoming company picnic',
 ' we should aim to be more efficient and look for ways to be more creative in our daily tasks',
 ' growth is essential for our future',
 ' but equally important is building strong relationships with our team members',
 ' as a reminder',
 ' the annual staff survey is due next friday',
 ' lastly',
 ' we agreed that we must take time to look over our plans carefully and consider all angles before moving forward',
 ' on a side note',
 ' david mentioned that his cat is recovering well from surgery',
 '"']

preprocess the phrases

the list of phrases is also in a txt file titles "lis.txt"

read from a txt file, and put them into a list of strings.

In [3]:
directory= os.getcwd()
wordsPath= directory + "/lis.txt"
standardized_phrases= [line.strip() for line in open(wordsPath, 'r')]
print(standardized_phrases)

['Optimal performance', 'Utilise resources', 'Enhance productivity', 'Conduct an analysis', 'Maintain a high standard', 'Implement best practices', 'Ensure compliance', 'Streamline operations', 'Foster innovation', 'Drive growth', 'Leverage synergies', 'Demonstrate leadership', 'Exercise due diligence', 'Maximize stakeholder value', 'Prioritise tasks', 'Facilitate collaboration', 'Monitor performance metrics', 'Execute strategies', 'Gauge effectiveness', 'Champion change']


prepare the sentence transformer

use a predefined model sentence transformer to check similarities.

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

create two dicts one to store the embedding of the input sentences and the other is to store the embedding of the standarized sentences

In [5]:
standardized_phrases_dict= {}
sentences_dict= {}
for phrase in standardized_phrases:
    standardized_phrases_dict[phrase]= model.encode(phrase, convert_to_tensor=True)

for sentence in sentences:
    sentences_dict[sentence]=model.encode(sentence, convert_to_tensor=True)


create the dict similarities to store the similarities between each phrase from the standarized phrases and each sentence form the input.

In [6]:
similarities= dict()
for key_stand,values_stand in standardized_phrases_dict.items():
    for key_sent, value_sent in sentences_dict.items():
        similarities[str(key_stand)+"__"+str(key_sent)]= util.pytorch_cos_sim(values_stand, value_sent)

sort the similarities in a descending order 
the dict sorted_similarities stores the similarities values between each phrase form the standarized phrases and each sentence form the input text.

In [7]:
sorted_similarities= {k: v for k, v in sorted(similarities.items(), key=lambda item: item[1], reverse=True)}

to show my results I will print only the keys and values when similarites greater than threshhold= 0.4 

P.S. you can change the threshhold and choose whatever number you want.

P.S2. I am prenting in the format """ 'standarized phrase__ input sentence', tnesor of similarities """

In [8]:
threshhold= 0.4
printable=[]
for key, value in sorted_similarities.items():
    if value > threshhold:
        printable +=(key, value)

printable

['Optimal performance__ we came to the consensus that we need to do better in terms of performance',
 tensor([[0.5740]]),
 'Enhance productivity__ we should aim to be more efficient and look for ways to be more creative in our daily tasks',
 tensor([[0.5527]]),
 'Drive growth__ growth is essential for our future',
 tensor([[0.5271]]),
 'Facilitate collaboration__ but equally important is building strong relationships with our team members',
 tensor([[0.4949]]),
 'Enhance productivity__ we came to the consensus that we need to do better in terms of performance',
 tensor([[0.4857]]),
 "Utilise resources__ it's important to make good use of what we have at our disposal",
 tensor([[0.4474]]),
 'Foster innovation__ growth is essential for our future',
 tensor([[0.4377]]),
 'Monitor performance metrics__ we came to the consensus that we need to do better in terms of performance',
 tensor([[0.4195]]),
 'Enhance productivity__ growth is essential for our future',
 tensor([[0.4113]]),
 'Conduct